In [1]:
from ediscs.Util import phot_tools
from ediscs.Util import error_analysis

import ediscs
import glob
import numpy as np
import os

If the SEXTRACTOR_PATH and EDISCS_REPO environment variables are not set, we need to set them.

In [2]:
required_keys = ['SEXTRACTOR_PATH', 'EDISCS_REPO']
for rk in required_keys:
    if rk not in os.environ.keys():
        print('Missing variable {}'.format(rk))
    else:
        print('{} set to {}'.format(rk, os.environ[rk]))

SEXTRACTOR_PATH set to /user/desjard/sextractor-2.19.5/config
EDISCS_REPO set to /user/desjard/programming/python/ediscs


If these are missing, you can set them with the following cell:

In [ ]:
# ONLY NEED TO RUN THIS IF VARIABLES ARE MISSING
# COMMENT OUT LINES AS NEEDED AND UPDATE PATHS
# AS NECESSARY

os.environ['SEXTRACTOR_PATH'] = '/path/to/sextractor/config_files/'
os.environ['EDISCS_REPO'] = os.path.dirname(ediscs.__file__)

Next, we need to set up the Python dictionary of FITS file names. This will be used to construct the SExtractor photometry script.

In [3]:
fits_files = glob.glob('cl1037_?_smooth.fits')
img_dict = {}

for f in fits_files:
    keyname = f.split('_')[1]
    img_dict[keyname] = f
    
print(img_dict)

{'z': 'cl1037_z_smooth.fits', 'b': 'cl1037_b_smooth.fits', 'v': 'cl1037_v_smooth.fits', 'r': 'cl1037_r_smooth.fits', 'i': 'cl1037_i_smooth.fits'}


Create the SExtractor photometry script and run SExtractor. You need to specify the key in the img_dict dictionary that will be the source detection image for SExtractor's dual-image mode. In this example, we've made the keys simple "b", "v", "r", etc., but they are generally arbitrary.

In [4]:
phot_tools.copy_sexfiles()
pscript = phot_tools.phot_script(img_dict, outfile='cl1037_pscript.sh', detection_img='r', clobber=True)
print('\nOur SExtractor script looks like this:\n')
print('\n'.join(pscript))


Our SExtractor script looks like this:

sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1037_z_smooth.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME "cl1037_z_smooth_bkgsub.fits" cl1037_r_smooth.fits cl1037_z_smooth.fits
sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1037_b_smooth.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME "cl1037_b_smooth_bkgsub.fits" cl1037_r_smooth.fits cl1037_b_smooth.fits
sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1037_v_smooth.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME "cl1037_v_smooth_bkgsub.fits" cl1037_r_smooth.fits cl1037_v_smooth.fits
sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1037_r_smooth.cat -CHECKIMAGE_TYPE "-BACKGROUND,SEGMENTATION" -CHECKIMAGE_NAME "cl1037_r_smooth_bkgsub.fits, cl1037_r_smooth_segmap.fits" cl1037_r_smooth.fits cl1037_r_smooth.fits
sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1037_i_smooth.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME "cl1

In [ ]:
!source 'cl1037_pscript.sh'

Now that photometry is done, we can get uncertainties. We do this on an image-by-image basis, so we'll just show how to do them for the V-band in this example. Each time, it will generate two plots: (1) histograms of the empty aperture photometry with their Gaussian fits, and (2) the error as a function of aperture size described by Labbé et al. (2003).

**IMPORTANT:** You must use the *background subtracted* image as the input for the empty aperture analysis. These have the same root name as the input FITS files with `_bkgsub.fits` appended to them. E.g., for an input of `cl1037_v_smooth.fits`, the background subtracted image will be `cl1037_v_smooth_bkgsub.fits`.

**REMEMBER:** If you have a K-band image, you also have an exposure map. This can be given as input to the Labbe error analysis code using `expmap="my_expmap.fits"` for example.

In [5]:
v_errors = error_analysis.Labbe(img_dict['v'].replace('.fits', '_bkgsub.fits'), img_dict['r'].replace('.fits', '_segmap.fits'), pixel_scale=0.25)
v_errors.do_photometry(3000)
error_fit = v_errors.fit_sigma_function(outroot='cl1037_v', label='CL1037 V')

CO1_1   =-1.27665130280501E-05                                                   [astropy.io.fits.card]
CO1_2   =-6.61113665999245E-05                                                   [astropy.io.fits.card]
CO1_4   =-2.82633373452844E-13                                                   [astropy.io.fits.card]
CO1_6   =-2.31699965665792E-12                                                   [astropy.io.fits.card]
CO2_4   =-6.61065574569778E-12                                                   [astropy.io.fits.card]
CO2_5   =-1.80852241783022E-12                                                   [astropy.io.fits.card]
CO2_6   =-5.69827286263806E-13                                                   [astropy.io.fits.card]


Checking if fake apertures are (1) in low exposure areas [optional] or (2) too close to a real source.
	Found 1694 out of 3000 good apertures.
Checking if fake apertures are too close to each other.
	Found 1686 out of 3000 good apertures. Using these results.
Doing aperture photometry...
	Working on r = 2.0 pixels (#1 out of 8)
	Working on r = 2.58342 pixels (#2 out of 8)
	Working on r = 3.33702 pixels (#3 out of 8)
	Working on r = 4.31046 pixels (#4 out of 8)
	Working on r = 5.56785 pixels (#5 out of 8)
	Working on r = 7.19204 pixels (#6 out of 8)
	Working on r = 9.29002 pixels (#7 out of 8)
	Working on r = 12.0 pixels (#8 out of 8)
Finished!
Saved Gaussian fits to histograms --> cl1037_v_gaussian_hist.png
Saved Labbe error function fit --> cl1037_v_errorfunc_fit.png


The warnings are *OKAY* (they're a result of Dennis Just's work) and don't affect what we're doing. Notice that the code printed out the number of good apertures that were actually used (we specified 3000 when we called `v_errors.do_photometry`, but only 1720 of the randomly generated positions were determined to be good). Running this will create our two PNG plots with file names dictated by the `outroot` optional keyword in the `Labbe()` class call. Be sure to check the plots that are produced to make sure that the fits to the data look reasonable. We've captured the output of the fit as the variable `error_fit`. Let's take a look now:

In [6]:
print(error_fit)

[0.6446755  4.10078078 0.37141054]


These numbers are the coefficients of the error function from Labbé et al. (2003) equation 3. You can get the uncertainty for any arbitrary aperture size using the `error_analysis.labbe_error_function()` function. This takes as input these three coefficients and also the *circularized radius* of your aperture. This is an important distinction. The aperture size is given by:

$N = \sqrt{\pi * A}$

where $N$ is the circularized radius and $A$ is the area of your aperture. This means you can use the area of any arbitrary aperture, such as a Kron aperture, to get the circularized radius and therefore the uncertainty in that aperture. The area $A$, and thus the circularized radius $N$, should be in units of pixels$^2$ and pixels, respectively. Let's say we have a Kron aperture with area 67.235 pixels$^2$. Thus, we can get the circularized radius and uncertainty:

In [7]:
kron_n = np.sqrt(np.pi * 67.235)
error_kron = error_analysis.labbe_error_function(kron_n, error_fit[0], error_fit[1], error_fit[2])
print('Circularized radius = {}'.format(kron_n))
print('Error in aperture = {}'.format(error_kron))

Circularized radius = 14.533581185107465
Error in aperture = 88.99759076731334


Finally, note that the error function is unique to each filter. You will need to do this for every filter you are interested in. Therefore, the above error terms only apply to the V-band image for CL1037.